In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import cv2 as cv2

In [3]:
df_train = pd.read_csv('Train.csv')
df_test = pd.read_csv('Test.csv')

df_train["is_train"] = 1
df_test["is_train"] = 0

df = pd.concat([df_train, df_test])

In [4]:
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12870 entries, 0 to 12869
Data columns (total 9 columns):
Filename    12870 non-null object
Width       12870 non-null int64
Height      12870 non-null int64
Roi.X1      12870 non-null int64
Roi.Y1      12870 non-null int64
Roi.X2      12870 non-null int64
Roi.Y2      12870 non-null int64
ClassId     12870 non-null int64
is_train    12870 non-null int64
dtypes: int64(8), object(1)
memory usage: 905.0+ KB


In [5]:
df.head()

,Filename,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,is_train
0,00000.ppm,53,54,6,5,48,49,16,1
1,00001.ppm,42,45,5,5,36,40,1,1
2,00002.ppm,48,52,6,6,43,47,38,1
3,00003.ppm,27,29,5,5,22,24,33,1
4,00004.ppm,60,57,5,5,55,52,11,1


In [6]:
df.ClassId.nunique(), df.ClassId.unique()

(43, array([16,  1, 38, 33, 11, 18, 12, 25, 35,  7, 23,  4,  9, 21, 20, 27,  3,
        13, 10,  5, 17, 34,  2,  8, 30, 24, 15, 26, 28, 22, 14, 32, 29,  6,
        36, 40, 41, 31, 19,  0, 39, 42, 37]))

In [38]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Activation, Conv2D, Flatten, Lambda
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.optimizers import Adam

tf.compat.v1.disable_eager_execution()

In [39]:
raw = df[['Filename', 'is_train', 'ClassId']]

In [40]:
raw = pd.get_dummies(raw, columns=['ClassId'])

In [41]:
raw_train = raw[(raw.is_train == 1)]
raw_test = raw[(raw.is_train == 0)]
X = raw_train.Filename
y = raw_train.iloc[:,2:]

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size= 0.1, shuffle=True)

In [43]:
y_valid.shape, y_train.shape

((1263, 43), (11367, 43))

In [44]:
def generator (samples, batch_size= 32, is_test= None):
    num_samples = len(samples)
    
    while 1:
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset + batch_size]
            
            images = []
            labels = []
            for batch_sample in batch_samples:
                local_image = cv2.cvtColor(cv2.imread(batch_sample[0]), cv2.COLOR_BGR2RGB)
                local_image = cv2.resize(local_image, (50,50))
                images.append(local_image)
                labels.append(batch_sample[1:])
                
            X_batch = np.array(images)
            y_batch = np.array(labels)
            
            if is_test:
                yield shuffle(X_batch)
            else:
                yield shuffle(X_batch, y_batch)

train_samples = pd.concat([X_train, y_train], axis= 1)
valid_samples = pd.concat([X_valid, y_valid], axis= 1)

train_samples = np.array(train_samples)
valid_samples = np.array(valid_samples)
batch_size = 4

train_generator = generator(train_samples, batch_size)
valid_generator = generator(valid_samples, batch_size)

In [45]:
raw_test_filtered = np.array( raw_test.drop(columns=['is_train']) )

In [46]:
model = Sequential()
model.add(Lambda(lambda x: x/255 - 0.5, input_shape=(50,50,3)))
model.add(Conv2D(3, (3,3), activation = 'relu'))
model.add(Conv2D(8, (3,3), strides= (2,2), activation = 'relu'))
model.add(Conv2D(32, (5,5), strides= (2,2), activation = 'relu'))
model.add(Conv2D(64, (5,5), strides= (2,2), activation = 'relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [47]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_2 (Lambda)            (None, 50, 50, 3)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 48, 48, 3)         84        
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 23, 23, 8)         224       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 10, 10, 32)        6432      
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 3, 3, 64)          51264     
_________________________________________________________________
flatten_2 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)              

In [48]:
model.compile(loss = 'categorical_crossentropy', optimizer= Adam(lr=0.0001), metrics=['accuracy'] )

In [ ]:
model.fit_generator(train_generator, epochs=5,  verbose = 1, steps_per_epoch=len(train_samples)//batch_size ,  validation_data = valid_generator, validation_steps = len(valid_samples)//batch_size )

In [ ]:
model.save('traffic_mode.h5')

In [ ]:
model.evaluate_generator( generator(raw_test_filtered, batch_size), steps=len(raw_test_filtered)/batch_size)